<a href="https://colab.research.google.com/github/shaghayegh5ghasemi/supervised_fuzzy_clustering/blob/main/Supervised_Fuzzy_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# load data
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=1, shuffle=True, num_workers=0)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
# plot the image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy() # convert from tensor
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

Files already downloaded and verified
Files already downloaded and verified


In [3]:
from IPython.core.display import Image

def make_window(img, label):
  windows = img.unfold(1, 3, 2).unfold(2, 3, 2) # unflod(dimension, size, step)
  selected_windows = []
  for n in range(2):
    i = torch.randint(0, 14, (1, 1))
    j = torch.randint(0, 14, (1, 1))
    temp = windows[:, i[0, 0], j[0, 0]].reshape((27, 1))
    selected_windows.append([temp, label])
  return selected_windows

def make_dataset():
  dataset = []
  for i, (inputs, labels) in enumerate(trainloader):
    if i == 1000:
      break
    dataset = dataset + make_window(inputs[0], labels[0])
  return dataset

In [4]:
dataset = make_dataset()

In [5]:
# initialize the centroids and the parameters
gamma = 1
sigma = 0.7
threshold = 0.0004
beta = 1

number_of_dimensions = 27
centroids = []
covariances = []
inverted_covariances = []

In [6]:
def calculate_mahanalobis_distance(icm, m, x):
  # D^2 = (x-m)^T * C^-1 * (x-m), mahanalobis distance formula
  s = x-m
  return float(s.T@icm@s)

In [7]:
def initialize_centroids(dataset, number_of_dimensions, centroids, inverted_covariances, covariances, gamma, sigma, threshold):
  number_of_centroids = len(centroids)
  for data in dataset:
    if number_of_centroids == 0:
      number_of_centroids += 1
      centroids.append(data)
      covariances.append(sigma*torch.eye(number_of_dimensions))
      inverted_covariances.append(torch.eye(number_of_dimensions)/sigma)
    else:
      distances = []
      for i in range(number_of_centroids):
        centroid = centroids[i]
        icm = inverted_covariances[i]
        distances.append(calculate_mahanalobis_distance(icm, centroid[0], data[0]))
      distances = -1*gamma*np.array(distances)
      RM = np.exp(distances)
      if max(RM) < threshold:
        number_of_centroids += 1
        centroids.append(data)
        covariances.append(sigma*torch.eye(number_of_dimensions))
        inverted_covariances.append(torch.eye(number_of_dimensions)/sigma)


initialize_centroids(dataset, number_of_dimensions, centroids, inverted_covariances, covariances, gamma, sigma, threshold)


In [8]:
def calculate_membership(icm, vi, x, gamma):
  distance = calculate_mahanalobis_distance(icm, vi, x)
  return np.exp(-1*gamma*distance)

In [9]:
def calculate_q_ij(centroids, inverted_covariances, classes, dataset, gamma):
  q = np.zeros((len(centroids), len(classes))) # i = number of clusters, j = number of classes
  for i in range(len(centroids)):
    centroid = centroids[i][0]
    icm = inverted_covariances[i]
    for j in range(len(classes)):
      numinator = 0
      denuminator = 0
      for data in dataset:
        vector = data[0]
        label = data[1]
        miu_ik = calculate_membership(icm, centroid, vector, gamma)
        if int(label) == j:
          numinator += miu_ik
        denuminator += miu_ik
      q[i][j] = numinator/denuminator
  return q

In [19]:
q = calculate_q_ij(centroids, inverted_covariances, classes, dataset, gamma)
print(q)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


[[3.32064368e-003 5.48408663e-001 1.22721407e-001 4.41460084e-002
  2.18306251e-002 2.91480475e-002 4.66350592e-003 1.15764757e-001
  1.01309125e-001 8.68721765e-003]
 [2.18776492e-002 3.63950790e-001 1.67949072e-001 4.26770802e-002
  2.29302422e-003 2.30932574e-001 1.10662295e-002 2.88188327e-003
  3.18147151e-002 1.24556982e-001]
 [2.37462467e-001 3.87438995e-002 1.06964947e-001 5.01836011e-002
  1.69393333e-004 4.82272245e-001 1.72102228e-020 1.10418828e-002
  4.27110404e-002 3.04505246e-002]
 [6.81927800e-001 1.59991344e-003 1.84452018e-001 3.05826084e-003
  1.36868716e-002 2.49796495e-002 5.58633956e-002 9.92199938e-003
  2.15616886e-002 2.94840337e-003]
 [6.03788307e-002 1.68447438e-005 3.09432927e-002 6.85001642e-001
  1.34614396e-002 2.51450277e-002 7.95445314e-003 1.98908273e-003
  1.70805397e-001 4.30398977e-003]
 [8.45442855e-002 9.96145879e-009 6.72418352e-001 3.48225298e-003
  1.81905351e-005 2.87689089e-003 1.04396543e-007 2.61517988e-004
  2.31032775e-001 5.36562077e-003

In [10]:
def update_centroids(centroids, inverted_covariances, classes, dataset, gamma, beta):
  q = calculate_q_ij(centroids, inverted_covariances, classes, dataset, gamma)
  for i in range(len(centroids)):
    centroid = centroids[i][0]
    icm = inverted_covariances[i]
    numinator = torch.zeros((27, 1))
    denuminator = 0
    for data in dataset:
      vector = data[0]
      label = data[1]
      miu_ik = calculate_membership(icm, centroid, vector, gamma)
      p_ik = q[i][int(label)]
      numinator += (miu_ik*p_ik)*vector
      denuminator += miu_ik*p_ik
    centroids[i][0] = (1-beta)*centroid + (beta/denuminator)*(numinator)

In [11]:
def calculate_inverted_covariance(covariance):
  covariance = np.array(covariance)
  l, mygamma = np.linalg.eig(covariance) # calculate eigenvalues and eigenvectors
  mylambda = np.diag(1/l) # construct a diagnosal matrix with eigenvalues
  icm = (mygamma)@(mylambda)@(mygamma.T) # invert of the covariance matrix
  return torch.tensor(icm)

In [16]:
def update_covariances(centroids, covariances, inverted_covariances, classes, dataset, gamma, beta):
  q = calculate_q_ij(centroids, inverted_covariances, classes, dataset, gamma)
  for i in range(len(centroids)):
    centroid = centroids[i][0]
    icm = inverted_covariances[i]
    numinator = torch.zeros((27, 27))
    denuminator = 0
    for data in dataset:
      vector = data[0]
      label = data[1]
      miu_ik = calculate_membership(icm, centroid, vector, gamma)
      p_ik = q[i][int(label)]
      numinator += (miu_ik*p_ik)*((vector-centroid)@(vector-centroid).T)
      denuminator += miu_ik*p_ik
    covariances[i] = (1-beta)*covariances[i] + (beta/denuminator)*(numinator)
    inverted_covariances[i] = calculate_inverted_covariance(covariances[i])

In [13]:
import math
def calculate_entropy(centroids, inverted_covariances, classes, dataset, gamma):
  q = calculate_q_ij(centroids, inverted_covariances, classes, dataset, gamma)
  ent = 0
  for i in range(len(centroids)):
    for data in dataset:
      vector = data[0]
      label = data[1]
      p_ik = q[i][int(label)]
      ent += -1*p_ik*math.log2(p_ik)
  return ent

In [17]:
for i in range(100):
  update_centroids(centroids, inverted_covariances, classes, dataset, gamma, beta)
  update_covariances(centroids, covariances, inverted_covariances, classes, dataset, gamma, beta)
  print(calculate_entropy(centroids, inverted_covariances, classes, dataset, gamma))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


ValueError: ignored

In [ ]:
# for i in range(100): 
#     if labels[i] == 9: 
#       print(images[i])
#       #imshow(torchvision.utils.make_grid(images[i]))
#       print(' '.join('%5s' % classes[labels[i]]))

imshow(torchvision.utils.make_grid(images[0]))
lists = make_window(img, labels[55])
# print(labels[55])
# imshow(torchvision.utils.make_grid(lists[55][0]))
#print(lists[55][1])
# print(lists[55][0])
# print(len(lists[55][0]))

for i in range(len(lists)): 
  imshow(torchvision.utils.make_grid(lists[i][0]))
  print(' '.join('%5s' % classes[lists[i][1]]))


imshow(torchvision.utils.make_grid(dataset[120][0]))
print(' '.join('%5s' % classes[dataset[120][1]]))

# get some random training images
# dataiter = iter(trainloader)
# images, labels = dataiter.next()

# patches = make_window(images[0], labels[0])

# for i in range(len(patches)):
#   print(patches[i][0].shape)